In [1]:
import os
from langgraph.prebuilt import create_react_agent
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_core.tools import tool, InjectedToolArg
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import folium
import numpy as np
import pandas as pd
from typing import List, Tuple
# from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor


from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent



import asyncio
import json
from collections.abc import Sequence
from typing import Any, cast
from urllib.parse import quote
from pathlib import Path
import requests
import aiohttp
# from mcp.server.fastmcp import FastMCP
# import mcp.server.stdio
# import mcp.types as types
# from mcp.server import NotificationOptions, Server
# from mcp.server.models import InitializationOptions
from langgraph.checkpoint.memory import MemorySaver
from langgraph_supervisor import create_supervisor

from scipy.spatial import cKDTree
from shapely.geometry import Point
import glob
# gradio
import folium.map
import gradio as gr
from gradio_folium import Folium

# Torch
from torch import nn
from torchvision.models.segmentation.deeplabv3 import DeepLabV3_ResNet50_Weights
import torchvision.models as models
from torchvision.io import decode_image
from torchvision.transforms import Resize
import torch

In [2]:
load_dotenv('./.env')
model = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=1,
    logprobs=True,
    seed = 12,
    top_logprobs=5,
)

### Initiate

In [3]:
# Create pandas dataframe
net_df = pd.read_csv("C:/Users/a940926/CHEN/Example-20250407T061515Z-001/Notebooks/net.csv")
tile_df = pd.read_csv("C:/Users/a940926/CHEN/Example-20250407T061515Z-001/Notebooks/tiles_polygons.csv")
veg_df = pd.read_csv("C:/Users/a940926/CHEN/Example-20250407T061515Z-001/Coordinates/loveda_final.csv")

# Convert WKT to geometry
net_df["geometry"] = net_df["geometry"].apply(wkt.loads)
tile_df["geometry"] = tile_df["geometry"].apply(wkt.loads)
veg_df["geometry"] = veg_df["geometry"].apply(wkt.loads)

# Create GeoDataFrames
net_gdf_original = gpd.GeoDataFrame(net_df, geometry="geometry", crs=4326)
tile_gdf = gpd.GeoDataFrame(tile_df, geometry="geometry", crs=4326)
veg_gdf = gpd.GeoDataFrame(veg_df,geometry="geometry", crs=4326)

In [4]:
class Deeplabv3(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
    
        self.deeplab = models.segmentation.deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.DEFAULT)
        # One class(forest/non-forest)
        self.deeplab.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))

        self.out = nn.Sigmoid()

    def forward(self, x):
        x1 = self.deeplab(x)['out']
        out = self.out(x1)
        return out

### Tools

In [5]:
# @tool(response_format="content_and_artifact")
@tool
def display_graph(id: str) -> str:
    """
    Function to display the vegetation distribution graph for a specific region based on the given id.
    """

    # Get the coordinates of the center of tile based on the id
    coordinates = tile_gdf[tile_gdf["name"]==f"output_{id}"].iloc[0,-1]
    lon = float(coordinates.split(",")[0][1:])
    lat = float(coordinates.split(",")[1][:-1])

    # Create a folium map centered at the coordinates
    global folium_map
    folium_map = folium.Map(location=[lat,lon], zoom_start=14)
    folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
           ).add_to(folium_map)
    

    # Plot the vegetation distribution for the given id
    veg = veg_gdf[veg_gdf["id"] == int(id)]
    folium_map = veg.explore(m=folium_map,color="green")

    print("Using tool display_graph with id:", int(id))
    return f"Map for id {id} displayed successfully."




# @tool(response_format="content_and_artifact")
@tool
def display_intersection_graph(id: str, distance: float = 5) -> str:
    """
    Function to calculate and plot the intersection between the vegetation and the high-voltage power line for a specific region based on the given id and distance.
    """

    # Get the coordinates of the center of tile based on the id
    coordinates = tile_gdf[tile_gdf["name"]==f"output_{id}"].iloc[0,-1]
    lon = float(coordinates.split(",")[0][1:])
    lat = float(coordinates.split(",")[1][:-1])


    # Create a folium map centered at the coordinates
    global folium_map
    folium_map = folium.Map(location=[lat,lon], zoom_start=14)
    folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
           ).add_to(folium_map)
    

    # Plot the intersection between the vegetation and the high-voltage power line
    veg = veg_gdf[veg_gdf["id"] == int(id)]
    veg = veg.to_crs(epsg=3857)
    net_gdf = net_gdf_original.to_crs(epsg=3857)
    net_gdf["buffer"] = net_gdf["geometry"].buffer(distance)

    print("Using tool display_intersection_graph with id:", int(id), "and distance:", f"{distance}m")
    try:
      intersection_gdf = gpd.overlay(gpd.GeoDataFrame({"geometry":veg["geometry"]}), gpd.GeoDataFrame({"geometry": net_gdf["buffer"]}), how="intersection")
      folium_map = veg.explore(color="green",m=folium_map)
      folium_map = intersection_gdf.explore(color="red",m=folium_map)
      folium_map = net_gdf.explore(color="yellow",m=folium_map)
      print(f"Intersection graph for id {id} with distance {distance}m displayed successfully.")
      return f"Intersection graph for id {int(id)} with distance {distance}m displayed successfully."
    except:
      return f"No intersection found for id {id} with distance {distance}m."
    



# @tool(response_format="content_and_artifact")
@tool
def display_risk_zone(id: str, distance: float = 5, num_of_riskzone: int = 3) -> str:
    """
    Function to calculate and plot the risk zone for a specific region based on the given id, distance and the number of risk zones asked.
    """

    # Get the coordinates of the center of tile based on the id
    coordinates = tile_gdf[tile_gdf["name"]==f"output_{id}"].iloc[0,-1]
    lon = float(coordinates.split(",")[0][1:])
    lat = float(coordinates.split(",")[1][:-1])


    # Create a folium map centered at the coordinates
    global folium_map
    folium_map = folium.Map(location=[lat,lon], zoom_start=14)
    folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
           ).add_to(folium_map)
    

    # Plot the risk zone and return its coordinates
    veg = veg_gdf[veg_gdf["id"] == int(id)]
    veg = veg.to_crs(epsg=3857)
    net_gdf = net_gdf_original.to_crs(epsg=3857)
    net_gdf["buffer"] = net_gdf["geometry"].buffer(distance)

    # Get the intersection between the vegetation and the high-voltage power line
    intersection_gdf = gpd.overlay(gpd.GeoDataFrame({"geometry":veg["geometry"]}), gpd.GeoDataFrame({"geometry": net_gdf["buffer"]}), how="intersection",keep_geom_type=False)
    intersection_gdf = intersection_gdf.to_crs(epsg=4326)
    veg = veg.to_crs(epsg=4326)
    net_gdf = net_gdf.to_crs(epsg=4326)
    # Calculate the area of the intersection geometries
    intersection_gdf["area"] = intersection_gdf.geometry.area
    # Sort the intersection geometries by area in descending order
    intersection_gdf = intersection_gdf.sort_values(by="area", ascending=False)
    # Get the centroids of the intersection geometries
    intersection_gdf["centroids"] = intersection_gdf["geometry"].centroid
    # Get the top 5 intersection geometries based on area
    intersection_gdf_top5 = intersection_gdf.head(num_of_riskzone)["centroids"]
    df = pd.DataFrame({"lat": [], "lon": []})
    for idx, element in enumerate(intersection_gdf_top5):
        lon= str(element).split("(")[-1].split(")")[0].split(" ")[0]
        lat = str(element).split("(")[-1].split(")")[0].split(" ")[1]
        new_row = pd.DataFrame({"lat": [lat], "lon": [lon]})
        df = pd.concat([df, new_row], ignore_index=True)
    print("Using tool display_risk_zone with id:", id, "distance:", f"{distance}m", "and number of riskzone:", num_of_riskzone)

    # Plot risk zones
    folium_map = veg.explore(color="green", m=folium_map)
    folium_map = intersection_gdf.explore(color="red", m=folium_map)
    folium_map = intersection_gdf.head(num_of_riskzone)["centroids"].explore(color="blue", m=folium_map)

    return df

@tool
def reverse_geocode_location(lat: float, lon: float)-> str:
    """Get the address given the lat and lon using Nominatim API."""
    url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&addressdetails=1&format=json"
    print("Using tool reverse_geocode_location with lat:", lat, "and lon:", lon)

    try:
        response = requests.get(url, headers={"User-Agent": "MCP-Geocoding-Tool/1.0 (Python)"})
        response_json = json.loads(response.content.decode('utf-8'))
        results = {}
        try:
            for key,element in response_json["address"].items():
                results[f"{key}"] = element
            return results
        except KeyError as e:
            return f"No address found for the given coordinates: ({lat}, {lon})."

    except aiohttp.ClientError as error:
        raise Exception(
            f"Network error: Unable to connect to geocoding service - {str(error)}"
        ) from error

@tool
def geocode_location(location: str, limit: int = 1) -> dict[str, Any]:
    """Get the coordinates given the name of the location using Nominatim API."""
    encoded_location = quote(location)
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={encoded_location}&limit={limit}&addressdetails=1"
    try:
      response = requests.get(url, headers={"User-Agent": "MCP-Geocoding-Tool/1.0 (Python)"})
      response_json = json.loads(response.content.decode("utf-8"))
      results = {}
      for key,element in response_json[0]["address"].items():
        results[key] = element
      results["lat"] = response_json[0]["lat"]
      results["lon"] = response_json[0]["lon"]
      return results
    except aiohttp.ClientError as error:
        raise Exception(
            f"Network error: Unable to connect to geocoding service - {str(error)}"
        ) from error

@tool
def get_id_for_nearest_location(lat: float, lon: float)-> str:
  """Get the id of the nearest location given the coordinates"""

  # Create a GeoDataFrame for the tile_gdf with the coordinates
  df = pd.DataFrame({"geometry":tile_gdf["position"].apply(lambda x: Point(x.split(",")[1][0:-1],x.split(",")[0][1:])), "name":tile_gdf["name"]})
  gdf = gpd.GeoDataFrame(df,geometry="geometry",crs=4326)
  df2 = pd.DataFrame({"geometry": [Point(lat,lon)]})
  gdf2 = gpd.GeoDataFrame(df2,geometry="geometry",crs=4326)

  nA = np.array(list(gdf.geometry.apply(lambda x: (x.x, x.y))))
  nB = np.array(list(gdf2.geometry.apply(lambda x: (x.x, x.y))))
  btree = cKDTree(nB)
  dist, idx = btree.query(nA, k=1)
  gdB_nearest = gdf2.iloc[idx].drop(columns="geometry").reset_index(drop=True)
  gdf = pd.concat(
        [
            gdf.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ],
        axis=1)

  return "The id of the nearest location to the given coordinate is: " + gdf.sort_values(by='dist').iloc[0]["name"].split("_")[-1]


# start a new conversation:
config = {"configurable": {"thread_id": "xyz" + str(np.random.randint(333))}}
memory = MemorySaver()

# Create an agent with the model and tools
agentTest = create_react_agent(
    model,
    checkpointer=memory,
    tools = [display_graph,display_intersection_graph, display_risk_zone,reverse_geocode_location,geocode_location, get_id_for_nearest_location],
    prompt=(
            "You are a helpful assistant to interact with the coordinates of vegetation and high-voltage network.\n\n"
            "Tool Usage Guidelines:\n"
            "1. To display the vegetation distribution graph, use the tool 'display_graph' with the given section/region/area id.\n"
            "2. To display the intersection between the vegetation and network, use the tool 'display_intersection_graph' with the given section/region/area id and distance\n"
            "3. To display the risk zone and return its coordinates, use the tool 'display_risk_zone' with the given section/region/area id and distance\n"
            "4. To get the address, use 'reverse_geocode_location'\n"
            "5. To get the coordinates given the address, use 'geocode_location'"
            "6. If being asked to get the id of the nearest location given the coordinates, use 'get_id_for_nearest_location'"
            )
            )


### Gradio app interface

In [6]:
import uuid
def chatbot_gradio(input_text, history, thread_id):
    # Generate a new thread_id if not provided
    config = {"configurable": {"thread_id": thread_id}}
    # Pass question to the langchain model and collect response in history to be compatible with gradio chat module
    response = agentTest.invoke({
        "messages": [
            {"role": "user", "content": input_text}
        ]
    }, config=config)

    for m in response["messages"]:
        # Print the response messages
        m.pretty_print()

    history.append((input_text, response['messages'][-1].content))
    return history, folium_map

def generate_thread_id():
    return str(uuid.uuid4())


#---------------------------------------------------------------
def find_all_files_in_directory(directory):
    """
    - Returns a list of all files in the specified directory
    Input: directory (str)
    Output: list of file paths (list of str)
    """
    file_paths = glob.glob(str(Path(directory) / "**") , recursive=True)
    return file_paths


def find_return_model_weights(root_path):
    """
    - Returns a list of model weights paths found in the specified directory
    - Passes the output to the textbox
    Input: directory (str)
    Output: list of model weights path (list of str)
    """
    model_weights_list = glob.glob(str(Path(root_path) / "*.pt"))
    return model_weights_list

def find_select_model_weights(root_path, prediction_mode):
    """
    - Returns a list (gr.Dropdown) of model weights paths found in the specified directory
    - If prediction_mode is 'mean', multiselect is enabled
    Input: directory (str), prediction mode (str)
    Output: list of model weights path (gr.Dropdown)
    """
    model_weights_list = glob.glob(str(Path(root_path) / "*.pt"))
    if prediction_mode == 'Ensemble':
        model_weights_list = ["All Models"]
        return gr.Dropdown(choices=model_weights_list, multiselect=True)
    else:
        model_weights_list = sorted(model_weights_list)
        return gr.Dropdown(choices=model_weights_list, multiselect=False)


# Predict 2
def predict(original_img, root_dir, model_weights_paths, prediction_mode, output_type):
    # Acelerator (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Get the original image size
    original_size = original_img.shape[:2]  # (height, width)
    # Prepare the input image for prediction 
    original_img = np.transpose(original_img, (2, 0, 1))  # Change from (H, W, C) to (C, H, W)
    img = torch.tensor(original_img, dtype=torch.float32)/255.0 # normalize and convert to tensor
    img_tensor = Resize((512, 512))(img).unsqueeze(0)  # Resize to 512x512 and add batch dimension
    
    # Initialize the model
    model = Deeplabv3()

    if prediction_mode == "Ensemble":
        # If the mode is "Ensemble", load all models and average their outputs
        outputs = []
        if model_weights_paths[0] == "All Models":
            # Find all model weights in the specified directory
            model_names = find_return_model_weights(root_dir)
            model_weights_paths = [Path(root_dir)/name for name in model_names]
        for weight_path in model_weights_paths:
            # Load the model weights
            model.load_state_dict(torch.load(weight_path, map_location=device)["model_state_dict"])
            model.eval()
            # Prediction
            with torch.no_grad():
                output = model(img_tensor) # Prediction
                output = Resize((original_size[1], original_size[0]))(output) # Resize the output to input size
                output = np.transpose(output.cpu().detach().numpy()[-1], (1, 2, 0)).squeeze(-1) # Remove the batch dimension
                if output_type == "Soft Mask":
                    # Soft : output is in range [0, 1], raw output from the model
                    outputs.append(output)
                else:
                    # Hard : output is binary (0 or 1)
                    outputs.append((output > 0.5).astype(np.uint8))  
        # Sum all outputs and normalize          
        outputs = np.sum(np.array(outputs),axis=0)
        maximum = np.max(outputs)
        outputs = outputs / maximum  # Normalize the outputs to [0, 1] 
        results = (np.stack(((outputs-1)*(-1),(outputs*0.5-1)*(-1) , (outputs-1)*(-1)), axis=2)*255).astype(np.uint8)

    elif prediction_mode == "Single Model":
        # If the mode is single, load only the selected model weights
        model.load_state_dict(torch.load(model_weights_paths, map_location=device)["model_state_dict"])
        model.eval()
        with torch.no_grad():
            # Prediction
            output = model(img_tensor)
            output = Resize((original_size[1], original_size[0]))(output) # Resize the output to input size
            output = np.transpose(output.cpu().detach().numpy()[-1], (1, 2, 0)).squeeze(-1) # Remove the batch dimension
            if not output_type == "Soft Mask":
                # Soft: output is in range [0, 1], raw output from the model
                # Hard: output is binary (0 or 1)
                output = np.where(output > 0.5, 1.0, 0)
            results = (np.stack(((output-1)*(-1), (output*0.5-1)*(-1), (output-1)*(-1)), axis=2)*255).astype(np.uint8)  # Convert to RGB format

    return results


In [7]:
with gr.Blocks(theme=gr.themes.Ocean()) as demo:
    gr.HTML("<h2 style='margin:0'>Altea Geoda</h2>")
    thread_id = gr.State()
    #Generate a new thread_id when the app loads
    demo.load(fn=generate_thread_id, inputs=[], outputs=thread_id)

    # Segmentation model tab
    prediction_modes = ["Single Model", "Ensemble"]
    with gr.Tab("Segmentation model"):    
        gr.HTML("<h2 style='margin:0'>Segmentation Model</h2>")

        #----------------------------------------------------------------------
        # Textbox for Input path
        Textbox_input_root_path = gr.Textbox(value = "C:/Users/a940926/CHEN/Example-20250407T061515Z-001/DEMO/model", label="Model Path", placeholder="Enter the path to the model weights directory")
        # Button 1: Find Model Weights
        b1 = gr.Button("Find Models",variant="primary")

        # Dropdown to select model weights (initially empty)
        Dropdown_select_model = gr.Dropdown(choices=[], label="Select Model", interactive=True)
        # Radio to choose prediction mode (single model or mean of multiple models)
        with gr.Row(variant="compact"):
            Radio_select_pred_mode = gr.Radio(label="Inference Mode", choices=prediction_modes, value='Single Model', type="value", interactive=True)
            Radio_select_output_type = gr.Radio(label="Output Type", choices=["Soft Mask", "Hard Mask"], value="Hard Mask", type="value", interactive=True)

        # When the Radio_select_pred_mode changes, update the dropdown with available model weights
        b1.click(fn=find_select_model_weights, inputs=[Textbox_input_root_path,Radio_select_pred_mode], outputs=Dropdown_select_model)
        Radio_select_pred_mode.change(fn=find_select_model_weights, inputs=[Textbox_input_root_path,Radio_select_pred_mode], outputs=Dropdown_select_model)
        #-----------------------------------------------------------------------

        with gr.Row(equal_height=True,variant="compact"):
            # Imagebox for input image
            img_input = gr.Image(type="numpy", label="Input Image", image_mode="RGB", width = 500, height = 500)
            img_output = gr.Image(type="numpy", label="Output Image", image_mode="RGB", width = 500, height = 500)
        b2 = gr.Button("Predict",variant="primary")
        b2.click(fn=predict, inputs=[img_input, Textbox_input_root_path,Dropdown_select_model, Radio_select_pred_mode, Radio_select_output_type], outputs=img_output)


    # Chatbot tab
    with gr.Tab("Chatbot"):
        gr.HTML("<h2 style='margin:0'>🤖 Chatbot</h2>")
        with gr.Row(equal_height=True):
            chatbot_component = gr.Chatbot(height=300)
            map1 = Folium(height=300)
        chat_input = gr.Textbox(placeholder="Write your message here", show_label=False)
        with gr.Row():
            chat_button = gr.Button("Send",variant="primary")
            new_conversation_button = gr.Button("New Conversation",variant="primary")
    
        chat_button.click(
            fn=chatbot_gradio,
            inputs=[chat_input, chatbot_component, thread_id],
            outputs=[chatbot_component, map1]
        )

    # Generate a new thread_id when the "New Conversation" button is clicked
        new_conversation_button.click(
            fn=generate_thread_id,
            inputs=[],
            outputs=thread_id
        ).then(
            fn=lambda: [],
            inputs=[],
            outputs=chatbot_component
        )

demo.launch()

C:\Users\a940926\AppData\Local\Temp\ipykernel_28228\2619163625.py:42: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_component = gr.Chatbot(height=300)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\a940926\AppData\Local\Temp\ipykernel_28228\4255131676.py:117: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection_gdf["area"] = intersection_gdf.geometry.area
C:\Users\a940926\AppData\Local\Temp\ipykernel_28228\4255131676.py:121: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection_gdf["centroids"] = intersection_gdf["geometry"].centroid


Using tool display_risk_zone with id: 57 distance: 100.0m and number of riskzone: 3
Using tool reverse_geocode_location with lat: 43.36858673582056 and lon: -3.8448574923295222
Using tool reverse_geocode_location with lat: 43.37103553201551 and lon: -3.843627118541633
Using tool reverse_geocode_location with lat: 43.37010988896269 and lon: -3.846534549263703
================================ Human Message =================================

Display the top3 risk zones with the largest intersection area for Maliaño, with a buffer distance of 100 meters. Return the addresses
================================== Ai Message ==================================
Tool Calls:
  geocode_location (call_SmaD1X83l4Dq42PpPL0hWXrR)
 Call ID: call_SmaD1X83l4Dq42PpPL0hWXrR
  Args:
    location: Maliaño
================================= Tool Message =================================
Name: geocode_location

{"railway": "Maliaño", "road": "Calle del Norte", "neighbourhood": "Barrio Buenos Aires", "village": "M

In [8]:
# with gr.Blocks() as demo:
#     with gr.Row():
#         # Left column with tabs
#         with gr.Column():
#             with gr.Tabs():
#                 with gr.Tab("Module 1"):
#                     gr.Markdown("### Left Module 1 Content")
#                 with gr.Tab("Module 2"):
#                     gr.Markdown("### Left Module 2 Content")
        
#         # Right column (always visible)
#         with gr.Column():
#             gr.Markdown("### Right Module (Always Visible)")
#             gr.Button("Right Button Example")

# demo.launch()

================================ Human Message =================================

Display intersection graph for Maliaño, with a buffer distance of 20 meters
================================== Ai Message ==================================
Tool Calls:
  geocode_location (call_Ed2UxthTi22XrbTNZZzbQoRU)
 Call ID: call_Ed2UxthTi22XrbTNZZzbQoRU
  Args:
    location: Maliaño
================================= Tool Message =================================
Name: geocode_location

{"railway": "Maliaño", "road": "Calle del Norte", "neighbourhood": "Barrio Buenos Aires", "village": "Maliaño", "state": "Cantabria", "ISO3166-2-lvl6": "ES-S", "ISO3166-2-lvl4": "ES-CB", "postcode": "39600", "country": "España", "country_code": "es", "lat": "43.4173678", "lon": "-3.8420976"}
================================== Ai Message ==================================
Tool Calls:
  get_id_for_nearest_location (call_epPJs5qaAFtE87WYJ04lh0Yw)
 Call ID: call_epPJs5qaAFtE87WYJ04lh0Yw
  Args:
    lat: 43.4173678
    lon